In [124]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import gpxpy
import os
from shapely.geometry import box, LineString
import matplotlib.pyplot as plt
import contextily as ctx
import time

In [125]:
links = []

with open("./Links.txt", mode="r", encoding="UTF-8") as f:
    for i in f:
        links.append(i.strip())

In [126]:
links

['https://caucasia.ru/gpx/8EWQAV62.gpx',
 'https://caucasia.ru/gpx/CPOQQN9R.gpx',
 'https://caucasia.ru/gpx/CVBOS4W2.gpx',
 'https://caucasia.ru/gpx/HTGT4UOT.gpx',
 'https://caucasia.ru/gpx/HQUWT5UB.gpx',
 'https://caucasia.ru/gpx/S87VDJ9R.gpx',
 'https://caucasia.ru/gpx/XKSRMH5L.gpx',
 'https://caucasia.ru/gpx/S87VDJ9R.gpx',
 'https://caucasia.ru/gpx/G6I1K7J9.gpx']

In [127]:
for num, url in enumerate(links):
    try:
        response = requests.get(url)
        filename = f"track{num}.gpx"
        
        with open(f"data/gpx/{filename}", mode="wb") as f:
            f.write(response.content)
    except Exception as e:
        print("Ошибка при скачивании")


In [128]:
gpx_list = os.listdir("data/gpx")

In [129]:
gpx_list

['track0.gpx',
 'track2.gpx',
 'track7.gpx',
 'track3.gpx',
 'track4.gpx',
 'track8.gpx',
 'track1.gpx',
 'track6.gpx',
 'track5.gpx']

In [130]:
margin = 0.02

In [131]:
for file in gpx_list:
    with open(f"{"data/gpx"}/{file}", mode="r", encoding="UTF-8") as f:
        gpx = gpxpy.parse(f)

    lats, lons = [], []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lats.append(point.latitude)
                lons.append(point.longitude)
    bbox = box(
            min(lons) - margin,
            min(lats) - margin,
            max(lons) + margin,
            max(lats) + margin
        )
    track_line = LineString(zip(lons, lats))

    gdf_bbox = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")
    gdf_bbox_web = gdf_bbox.to_crs(epsg=3857)

    gdf_track = gpd.GeoDataFrame(geometry=[track_line], crs="EPSG:4326")
    gdf_track_web = gdf_track.to_crs(epsg=3857)

    _, ax = plt.subplots(figsize=(10, 8))

    gdf_bbox_web.plot(ax=ax, alpha=0)
    gdf_track_web.plot(ax=ax, color="red", linewidth=2)

    ctx.add_basemap(ax, crs=gdf_bbox_web.crs, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()
    os.makedirs("data/image", exist_ok=True)

    plt.savefig(f"{"data/image"}/{file[:-4]}", dpi=150, bbox_inches="tight", pad_inches=0)
    plt.close()


https://nominatim.openstreetmap.org

In [132]:
len(lats)

673

In [133]:
len(lons)

673

In [134]:
names_json = []

In [135]:
ll = list(zip(lats, lons))


In [136]:
for lat, lon in (ll[0], ll[len(ll)//5],  ll[len(ll)//2], ll[len(ll)//3],  ll[-1]):
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json", headers=headers)
            print(response)
            json = response.json()
            time.sleep(3)

            names_json.append(json["display_name"])
        except Exception as e:
            print(f"Возникла ошибка при получении данных с API: {e}")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [137]:
names_json

['Поляна, Голубинское сельское поселение, Бахчисарайский район, Автономна Республіка Крим, 298474, Україна',
 'Исчезает, Поляна, Голубинское сельское поселение, Бахчисарайский район, Автономна Республіка Крим, 298474, Україна',
 'На Бельбекскую долину, Одичавшая, Голубинское сельское поселение, Бахчисарайский район, Автономна Республіка Крим, 298475, Україна',
 'Одичавшая, Голубинское сельское поселение, Бахчисарайский район, Автономна Республіка Крим, 298475, Україна',
 'Поляна, Голубинское сельское поселение, Бахчисарайский район, Автономна Республіка Крим, 298474, Україна']